# Hyper-parameter tuning with Smart Search

All hyper-parameters that influence the learning are searched simultaneously (except for the number of estimators, which poses a time / quality tradeoff).

## Notebook variables

(Can be accessed by papermill)

In [ ]:
EVALS = 4 # the number of evaluations roughly determines how long search procedures will take; this value is just for tests, it should be increased for meaningful results!
DATASET = "digits" # "digits" (rather small, good to start running experiments) or "credit" (bigger)
FOLDS = 3 # smaller values will make the evaluation quicker; recommended values are between 3 and 10
NTREES = 10 # reasonable values between 10 and 100; this has an impact on the time an evaluation takes
ESTIMATOR = "rf" # "rf" or "xgb"
HP_DIST = { # this is for rf
    "max_depth": {
        "type": "int",
        "min": 2,
        "max": 11
    },
    "max_features":{
        "type": "float",
        "min": 0.1,
        "max": 1.0
    },
    "bootstrap": {
        "type": "choice",
        "values": [True, False]
    }
}
SCORING = "neg_log_loss" # negative log likelihood
SEED = 42 # for reproducibility of results

## Default model

Initialize an empty model with specified number of trees (`NTREES`) and random state (`SEED`), and with default hyper-parameter values.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

if (ESTIMATOR=="rf"):
    default_params = {"n_estimators": NTREES, "random_state": SEED}
    default_model = RandomForestClassifier(**default_params)
elif (ESTIMATOR=="xgb"):
    default_params = {"n_estimators": NTREES, "seed": SEED}
    default_model = XGBClassifier(**default_params)
    if (DATASET=="credit"):
        default_params.update({"objective": "multi:softprob"})

## Data

Import data and prepare X and y (inputs and outputs)

In [ ]:
if (DATASET=="digits"):
    from sklearn.datasets import load_digits
    digits = load_digits()
    X, y = digits.data, digits.target
elif (DATASET=="credit"):
    from mlxtend.data.kaggle_gmsc import kaggle_gmsc_data_nomissing
    X, y = kaggle_gmsc_data_nomissing()

## Hyper-parameter distributions

Create hyperopt-compatible distributions based on values stored in `HP_DIST`.

In [ ]:
from hyperopt import hp     
hp_space = {}
for v in HP_DIST.keys():
    if HP_DIST[v]['type']=="int":
        hp_space[v] = hp.quniform(v, HP_DIST[v]['min'], HP_DIST[v]['max'], 1)
    elif HP_DIST[v]['type']=="float":
        hp_space[v] = hp.uniform(v, HP_DIST[v]['min'], HP_DIST[v]['max'])
    elif HP_DIST[v]['type']=="choice":
        hp_space[v] = hp.choice(v, HP_DIST[v]["values"])

Hyperopt uses float, but when parameters are actually int, scikit needs them to be int!

Let's create a function that "converts" hyperopt-compatible parameters to scikit-compatible parameters:

In [ ]:
def hyperopt2scikit_params(h_params):
    s_params = h_params
    for v in HP_DIST.keys():
        if HP_DIST[v]['type']=="int":
            s_params[v] = int(h_params[v])
    s_params.update(default_params)
    return s_params

## Define loss to minimize

Hyperopt aims at minimizing the loss incurred by using a certain set of hyperparameter values. We define the loss function to use, which implements the evaluation of these hyperparameter values and requires access to the data.

In [ ]:
from hyperopt import STATUS_OK
from sklearn.model_selection import cross_val_score

def hp_loss(hp_params):
    s_params = hyperopt2scikit_params(hp_params)
    if (ESTIMATOR=="xgb"):
        model = XGBClassifier(**s_params)
    elif (ESTIMATOR=="rf"):
        model = RandomForestClassifier(**s_params)
    s = cross_val_score(model, X, y, scoring=SCORING, cv=FOLDS, n_jobs=-1, verbose=1)
    return {'loss': -s.mean(), 'status': STATUS_OK}

## Run minimization

With hyperopt, we pass the hyper-parameter distributions to use, along with the loss function to minimize.

Note the differences in design compared to scikit's RandomizedSearch (where we just pass the data via a `fit` function).

In [ ]:
from hyperopt import fmin, tpe, Trials
from numpy.random import RandomState

trials = Trials() # this is where the history will be saved
%time best = fmin(hp_loss, hp_space, algo=tpe.suggest, trials=trials, max_evals=EVALS) # TODO: specify seed

In [ ]:
hyperopt2scikit_params(best)

(Note on using `neg_log_loss` as our performance metric: we expect to see loss values which are > 0 since we're dealing with negative log likelihood (likelihood is between 0 and 1 and log makes those values negative); the smaller the loss, the higher the likelihood, the better!)

Save results

In [ ]:
from joblib import dump
dump(trials, "output/smart_search_" + DATASET + ".joblib")

Review results

In [ ]:
trials.losses()

In [ ]:
trials.best_trial

In [ ]:
trials.best_trial['misc']['vals']